Imports

In [1]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
from QLCstuff2 import getNQLL
import reference_solution as refsol
from scipy.fft import rfft  #, irfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Data preparation/pre-processing

In [2]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
D = GI['D']
D_units = GI['D_units']
D = AssignQuantity(D,D_units)# Compute Nqll_eq
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1

# Define initial conditions
Ntot_init_1D = np.ones(nx_crystal)
Nqll_init_1D = getNQLL(Ntot_init_1D,NSTAR,NBAR)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = np.column_stack((x.flatten(), t.flatten())).to(device)

In [3]:
def f1d_solve_ivp_dimensionless(Ntot, Nqll, dNqll_dxx, scalar_params, sigmaI):
    """
    Adapted from QLCstuff2, this function computes the right-hand side of
    the two objective functions that make up the QLC system.
    
    Returns:
        [dNtot_dt, dNqll_dt]
    """
    sigma0, omega_kin = scalar_params

    # Diffusion term based on FT
    # Dcoefficient1 = np.pi**2 / (L)**2  
    # bj_list = rfft(Nqll)
    # cj_list = bj_list*J2_LIST
    # dy = -Dcoefficient1 * irfft(cj_list)    # This is actually a second derivative...

    # Ntot deposition
    m = (Nqll - (NBAR - NSTAR))/(2*NSTAR)
    sigma_m = (sigmaI - m * sigma0)
    dNtot_dt = omega_kin * sigma_m
    dNtot_dt += dNqll_dxx

    # NQLL    
    dNqll_dt = dNtot_dt - (Nqll - (NBAR - NSTAR*tf.sin(2*np.pi*Ntot)))
    
    # Package for output
    return dNtot_dt, dNqll_dt

In [ ]:
def QLC_model(xs, ys):
    """Defines QLC model. Acts as collocation point loss function.

    Args:
        xs: xs[0] = x, xs[1] = t
        ys: ys[0] = Ntot, ys[1] = Nqll

    Returns:
        [Ntot-loss, Nqll-loss]

    """
    # TODO - rewrite to make it so I don't load all this for each epoch?
    Ntot, Nqll = ys[:, 0:1], ys[:, 1:]
    
    # Compute gradients
    dNtot_dt = dde.grad.jacobian(ys, xs, i=0, j=1)
    dNqll_dt = dde.grad.jacobian(ys, xs, i=1, j=1)
    dNqll_dxx = dde.grad.hessian(ys, xs, i=1, j=0)

    # Supersaturation reduction at center
    c_r = GI['c_r']

    # Thickness of monolayers
    h_pr = GI['h_pr']
    h_pr_units = GI['h_pr_units']
    h_pr = AssignQuantity(h_pr,h_pr_units)
    h_pr.ito('micrometer')

    # Deposition velocity
    nu_kin = GI['nu_kin']
    nu_kin_units = GI['nu_kin_units']
    nu_kin = AssignQuantity(nu_kin,nu_kin_units)

    # Difference in equilibrium supersaturation between microsurfaces I and II
    sigma0 = GI['sigma0']

    # Supersaturation at facet corner
    sigmaI_corner = GI['sigmaI_corner']

    # Time constant for freezing/thawing
    tau_eq = GI['tau_eq']
    tau_eq_units = GI['tau_eq_units']
    tau_eq = AssignQuantity(tau_eq,tau_eq_units)

    # Compute omega_kin
    nu_kin_mlyperus = nu_kin/h_pr
    nu_kin_mlyperus.ito('1/microsecond')
    omega_kin = nu_kin_mlyperus.magnitude * tau_eq.magnitude

    # Compute sigmaI
    sigmaI = sigmaI_corner*(c_r*(X_QLC/L)**2+1-c_r)
    
    # Nbar, Nstar, sigma0, omega_kin, deltax = scalar_params
    scalar_params = np.asarray([sigma0, omega_kin])

    dNtot_dt_rhs, dNqll_dt_rhs = f1d_solve_ivp_dimensionless(Ntot, Nqll, dNqll_dxx, scalar_params, sigmaI)

    # Return [Ntot-loss, Nqll-loss]
    return [dNtot_dt - dNtot_dt_rhs, # dNtot_dt = Nqll*surface_diff_coefficient + w_kin*sigma_m
            dNqll_dt - dNqll_dt_rhs] # dNqll_dt = dNtot/dt - (Nqll - Nqll_eq)

Define IcePINN class

In [5]:
# Difference between nn._ and nn.functional._ is that functional is stateless: 
# https://stackoverflow.com/questions/63826328/torch-nn-functional-vs-torch-nn-pytorch
class IcePINN(nn.Module):
    def __init__(self, num_hidden_layers, hidden_layer_size):
        super().__init__()
        self.fc_in = nn.Linear(2, hidden_size)
        self.fc_hidden = nn.ModuleList()
        for _ in range(num_hidden_layers-1):
            self.fc_hidden.append(nn.Linear(hidden_size, hidden_size))
        self.fc_out = nn.Linear(hidden_size, 2)

    def forward(self, x):
        x = F.tanh(self.fc_in(x))
        for layer in self.fc_hidden:
            x = F.tanh(layer(x))
        x = F.linear(self.fc_out(x))
        return x

Instantiate model and optimizers

In [ ]:
model = IcePINN(num_hidden_layers=8, hidden_layer_size=80).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Training loop

In [ ]:
def train_PINN(model: IcePINN, optimizer, epochs, training_set):

for epoch in range(epochs):
    
    
    # model predicts output of training_set as batch
    pred = model.forward(training_set)

    # evaluate collocation point loss
    

    # backward and optimize
    loss.backward() # Computes loss gradients
    optimizer.step() # Adjusts weights accordingly
    optimizer.zero_grad() # Zeroes gradients so they don't affect future computations




Save model

In [ ]:
print('Finished Training')
PATH = './models/TestPINN.pth'
torch.save(model.state_dict(), PATH)

Evaluate Model (visually?)

In [ ]:
loaded_model = IcePINN()
loaded_model.load_state_dict(torch.load(PATH)) # it takes the loaded dictionary, not the path file itself
loaded_model.to(device)
loaded_model.eval()